# 0: import

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Flatten, Dense, Input
from keras.models import Model
from keras.regularizers import l2

import numpy as np

Using TensorFlow backend.


# 1: base model

In [2]:
def model(input_shape=[224,224,3], num_labels=5, reg=5e-2):
    X = Input(shape=input_shape)
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=X)
    fl = Flatten()(base_model.output)
    predict = Dense(units=num_labels, activation='sigmoid', kernel_regularizer=l2(reg), kernel_initializer='he_uniform')(fl)
    
    _m = Model(inputs=X, outputs=predict)
    _m.compile(loss='binary_crossentropy', optimizer='Adam')
    
    return _m

In [3]:
M = model()
# M.summary()

# 2: Train

In [4]:
def preprocess_image(img_path):
    x = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    return x

## 2.0: gen img

In [5]:
# datagen = image.ImageDataGenerator(
#     rotation_range=120,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.3,
#     zoom_range=0.3,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

In [6]:
# def gen_img(x, k, path='./preview', pref='_'):
#     i = 0
#     for _ in datagen.flow(x, batch_size=1, save_to_dir=path, save_prefix=pref, save_format='jpeg'):
#         i += 1
#         if i > k:
#             break

In [7]:
# import os
# cc = ['./data/potato/', './data/carrot/', './data/egg/', './data/Mushroom/', './data/pasta_sause/']
# for i, c in enumerate(cc):
#     for img_name in os.listdir(c):
#         x = preprocess_image(c+img_name)
#         gen_img(x, 20, c)

In [8]:
# img = image.load_img('./all.JPG', target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# gen_img(x, 20)

## 2.1: load images

In [9]:
import os
cc = [
    './data/potato/', './data/carrot/', './data/egg/', './data/Mushroom/', './data/pasta_sause/'
]
train_prep = []
train_label = []
for i, c in enumerate(cc):
    for img_name in os.listdir(c):
        x = preprocess_image(c+img_name)
        train_prep.append(x)
        train_label.append(i)

In [10]:
train_df = np.squeeze(np.array(train_prep), axis=1)
train_df = preprocess_input(train_df)

In [11]:
train_targets = np.zeros((train_df.shape[0], len(cc)))
train_targets[np.arange(train_df.shape[0]), train_label] = 1

## 2.2: fit

In [12]:
M.fit(x=train_df, y=train_targets, batch_size=16, epochs=20, verbose=1, shuffle=True)

Epoch 1/20
1023/1023 [==============================] - 81s - loss: 0.6276    
Epoch 2/20
1023/1023 [==============================] - 74s - loss: 0.2229    
Epoch 3/20
1023/1023 [==============================] - 74s - loss: 0.1249    
Epoch 4/20
1023/1023 [==============================] - 74s - loss: 0.0892    
Epoch 5/20
1023/1023 [==============================] - 74s - loss: 0.0948    
Epoch 6/20
1023/1023 [==============================] - 74s - loss: 0.0840    
Epoch 7/20
1023/1023 [==============================] - 74s - loss: 0.0901    
Epoch 8/20
1023/1023 [==============================] - 74s - loss: 0.0921    
Epoch 9/20
1023/1023 [==============================] - 74s - loss: 0.0838    
Epoch 10/20
1023/1023 [==============================] - 74s - loss: 0.0506    
Epoch 11/20
1023/1023 [==============================] - 74s - loss: 0.0445    
Epoch 12/20
1023/1023 [==============================] - 74s - loss: 0.0453    
Epoch 13/20
1023/1023 [==========================

# 3: predict

In [15]:
x = preprocess_image('./egg_pot.JPG')
x = preprocess_input(x)
preds = M.predict_on_batch(x)
preds

array([[  1.72642712e-03,   4.00346908e-04,   9.99368966e-01,
          1.81403209e-03,   9.12638090e-04]], dtype=float32)

# 4: save model weights

In [14]:
# M.save('./Model_Weights.h5')